In [8]:
# Supondo que os módulos agent.py e groupchat.py estejam disponíveis
from groupchat import GroupChat
from agent import Agent
from groupchatadmin import GroupChatAdmin
from pipeline import PipelineComponent
from pipeline import Pipeline

# Definindo um pipeline fictício
class HumanInputMessage:
    def run(self, data):
        print(data)
        resposta = input("Digite a resposta: ")
        return resposta
    
class HumanInputPipeline(PipelineComponent):
    def process(self, **kwargs):
        chatgroup = kwargs.get('chatgroup')
        if not chatgroup:
            raise ValueError("Chatgroup is required for HumanInputPipeline")
        
        agents = chatgroup.agentList
        for i, agent in enumerate(agents):
            print(f"{i + 1}: {agent.name}")

        escolha = None
        while escolha is None:
            try:
                escolha_input = input("Digite o número correspondente ao agente: ")
                escolha = int(escolha_input)
                if escolha < 1 or escolha > len(agents):
                    print("Escolha inválida. Tente novamente.")
                    escolha = None
            except ValueError:
                print("Entrada inválida. Por favor, digite um número.")
        
        next_agent = agents[escolha - 1]
        kwargs['next'] = next_agent
        return kwargs

class AgentReplyComponent(PipelineComponent):
    def process(self, **kwargs):
        # Recupera o agente do kwargs
        agent = kwargs.get('next')
        group_chat = kwargs.get('chatgroup')
        messages = group_chat.get_messages()
        if not agent:
            raise ValueError("Agent is required for AgentReplyComponent")

        # # Recupera os dados para a resposta
        # if not messages:
        #     raise ValueError("Messages is required for AgentReplyComponent")

        # Chama generate_reply do agente e armazena a resposta
        reply = agent.generate_reply(data=messages)
        print(reply)
        group_chat.add_message(sender_id=agent.agent_id, message=reply)


pipeline_admin = Pipeline([HumanInputPipeline(), AgentReplyComponent()])

# Setup do ambiente de teste
group_chat = GroupChat()
agent1 = Agent("agent1", "Agente 1", "role1", DummyPipeline())
agent2 = Agent("agent2", "Agente 2", "role2", DummyPipeline())

group_chat.add_agent(agent1)
group_chat.add_agent(agent2)

chat_admin = GroupChatAdmin("admin", "Admin", "admin_role", pipeline_admin, group_chat, "manage_chat")

# Executando o GroupChatAdmin
chat_admin.run()


1: Agente 1
2: Agente 2
Empty DataFrame
Columns: [id, sender_id, message, timestamp]
Index: []
ola
1: Agente 1
2: Agente 2
   id sender_id message                   timestamp
0   1    agent1     ola  2023-12-26 06:40:49.989950
ola 2
1: Agente 1
2: Agente 2
   id sender_id message                   timestamp
0   1    agent1     ola  2023-12-26 06:40:49.989950
1   2    agent2   ola 2  2023-12-26 06:41:03.019843
tudo bem?
1: Agente 1
2: Agente 2
   id sender_id    message                   timestamp
0   1    agent1        ola  2023-12-26 06:40:49.989950
1   2    agent2      ola 2  2023-12-26 06:41:03.019843
2   3    agent1  tudo bem?  2023-12-26 06:41:09.942985


KeyboardInterrupt: Interrupted by user

In [9]:
group_chat.get_messages()

,id,sender_id,message,timestamp
0,1,agent1,ola,2023-12-26 06:40:49.989950
1,2,agent2,ola 2,2023-12-26 06:41:03.019843
2,3,agent1,tudo bem?,2023-12-26 06:41:09.942985
